In [219]:
!pip install spotipy
import pandas as pd
import numpy as np
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pickle
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt




sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
    client_id="63179a6edbf84ad186533d354617f71b",
    client_secret="f2ed34108bf04f199f1850809d1750d0"))

In [220]:
songs = pd.read_csv('song_id_extraction.csv')

In [221]:
songs

,song_id
0,001McWPVLHQbQzDVq2Ndiq
1,003vvx7Niy0yvhvHt4a68B
2,0071kWo4vzvKWe6dsxKeWj
3,00B7SBwrjbycLMOgAmeIU8
4,00CqEmnPLFKDhAb3cuu6Cs
...,...
20143,7zvYcp4GcbqkXWCL03pkO8
20144,7zwwvrJAWGjfc9wFD3bVzZ
20145,7zxUQuM8AKfuXLatnmWuWm
20146,7zxa7uJdNakO0braiv8jdx


In [224]:
import time

song_ids = songs['song_id'].tolist()

audio_features = []
for id in song_ids:
    track_features = sp.audio_features(id)
    audio_features.extend(track_features)
    
    time.sleep(10)  

Max Retries reached


SpotifyException: http status: 429, code:-1 - /v1/audio-features/?ids=001McWPVLHQbQzDVq2Ndiq:
 Max Retries, reason: too many 429 error responses

In [ ]:
df_features = pd.DataFrame(audio_features)

In [ ]:
df_features.head()

In [ ]:
songs_features=pd.concat([songs,df_features],axis=1)
songs_features

In [ ]:
songs_features=songs_features.dropna()
songs_features

In [ ]:
X = songs_features[['danceability','energy','key','loudness','mode','speechiness','acousticness','instrumentalness','liveness','valence','tempo']]

In [ ]:
#scaler (numeric)

scaler =MinMaxScaler()
scaler.fit(X)


X_prep = scaler.transform(X)

scaler_filename ='scaler_sd.pkl'

pickle.dump(scaler, open(scaler_filename, 'wb'))


In [ ]:
X_df = pd.DataFrame(X_prep)
X_df

In [ ]:
#Finding Epsilon
k = len(X_df.columns) + 1 #best practice is to add as k the length of the dataset + 1
nearest_neighbors = NearestNeighbors(n_neighbors=k)
neighbors = nearest_neighbors.fit(X_prep)
distances,indices =neighbors.kneighbors(X_prep)
distances = np.sort(distances[:,-1],axis=0)
plt.plot(distances)
plt.show()

In [ ]:
model = DBSCAN(eps=0.9,min_samples=k)
clusters = dbscan.fit_predict(X_prep)

X_df['clusters']=clusters

model_filename = 'dbscan.pkl'

pickle.dump(model, open(model_filename, 'wb'))

X_prep = scaler.transform(X_prep)

In [ ]:
X_df['clusters'].value_counts()


In [225]:
X_df.head()

,0,1,2,3,4,5,6,7,8,9,10,clusters
0,0.091605,0.724780,0.000000,0.651390,1.0,0.036222,0.457468,0.070695,0.640058,0.412008,0.506422,0
1,0.344220,0.908958,0.090909,0.876688,1.0,0.055264,0.001253,0.000000,0.077455,0.244306,0.603768,0
2,0.225568,0.986396,0.090909,0.960435,1.0,0.161742,0.012238,0.000000,0.250206,0.514493,0.801519,0
3,0.741005,0.698619,1.000000,0.867275,0.0,0.039068,0.007726,0.005629,0.126444,0.504141,0.419466,1
4,0.628732,0.986396,0.181818,1.000000,0.0,0.054826,0.041491,0.000000,0.140883,0.493789,0.385548,1
